In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle

### read in files

In [2]:
datadir = '../../preprocess/kleinlab/output_subset/'

In [3]:
df_metadata = pd.read_csv(os.path.join(datadir,'metadata.tsv.gz'),sep='\t',index_col=0)
print(df_metadata.shape)
df_metadata.head()

(3221, 4)


,time,Population,Annotation,Well
84,6.0,LK,Mast,2
115,6.0,LK,Monocyte,2
190,6.0,LK,Mast,2
205,6.0,LK,Mast,2
337,6.0,LK,Meg,2


In [4]:
df_coord = pd.read_csv(os.path.join(datadir,'coordinates.tsv.gz'),sep='\t',index_col=0)
print(df_coord.shape)
df_coord.head()

(3221, 2)


,0,1
84,-1674.817,-867.362
115,2970.137,390.955
190,-1667.094,-846.103
205,-1769.354,-902.560
337,-1866.029,739.786


In [5]:
df_clones = pd.read_csv(os.path.join(datadir,'clones.tsv.gz'),sep='\t',index_col=0)
print(df_clones.shape)
df_clones.head()

(3221, 5864)


,0,1,2,3,4,5,6,7,8,9,...,5854,5855,5856,5857,5858,5859,5860,5861,5862,5863
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
### remove clones that have zero cells
df_clones = df_clones[df_clones.columns[df_clones.sum(axis=0)>0]]
print(df_clones.shape)
df_clones.head()

(3221, 365)


,50,82,105,111,134,190,213,239,251,269,...,5744,5764,5772,5773,5774,5805,5806,5818,5832,5850
84,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### define function to calculate clone pair-wise distance

In [7]:
### This is the function you should write
### here I am importing the one I wrote so you can have a reference

from megatron_HC import cal_clone_dist
st=time.time()
list_dist = cal_clone_dist(df_clones,df_coord,df_metadata)
ed = time.time()
print('running time(mins): '+ str((ed-st)/60))

ModuleNotFoundError: No module named 'seaborn'

`list_dist` stores the clone-pair distance. The order should be consistent with the column of `df_clones`

E.g.  given clones `[50,82,105,111]`,
the order of clone pairs in `list_dist` should be:

`[(50, 82), (50, 105), (50, 111), (82, 105), (82, 111), (105, 111)]`

In [ ]:
list_dist

`cal_clone_dist()` might take some time. The result can be saved as a `.pkl` file

```python
#save list_dist
with open('list_dist.pkl', 'wb') as f: 
    pickle.dump(list_dist, f)
#read back list_dist
with open('list_dist.pkl', 'rb') as f:
    list_dist = pickle.load(f)
```

### validation

#### comparing clone clusters with ground truth

In [12]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics.cluster import adjusted_rand_score

In [ ]:
Z = linkage(list_dist, 'ward')
# clone_clusters = fcluster(Z, 5000, criterion='distance')
clone_clusters = fcluster(Z, 5, criterion='maxclust')

In [ ]:
clone_clusters

In [8]:
### read in groundtruth file
df_truth = pd.read_csv(os.path.join(datadir,'./clones_truth.tsv.gz'),sep='\t',index_col=0)
### score the result
#adjusted_rand_score(df_truth['celltype'].tolist(), clone_clusters.tolist())

## Arya's Stuff

In [50]:
%run megatron_ak.ipynb

In [19]:
# read in data
cellid_to_idx, cloneid_to_idx, clonegrouping_spmtx, coords_mtx, time_vec = readin(
    "/home/luak/rotations/pinello/post/MEGATRON/preprocess/kleinlab/output_subset"
)

Number of cells:  3221
Number of clones:  5864
Number of dimensions:  2
Time Steps:  [2. 4. 6.]


### Wasserstein Trials

In [20]:
linkage_options = ["ward", "single", "average", "complete", "weighted", "centroid", "median"]
wassdist_options = ["wasserstein", "energy"]
#criterion_options = ["inconsistent", "distance", "maxclust", "monocrit", "maxclust_monocrit"]
criterion_options = ["inconsistent", "distance", "maxclust"]

score_dict = {}

# wasserstein options
for wd_o in wassdist_options:
    dist_mat = getdistance_wasserstein(clonegrouping_spmtx, coords_mtx, time_vec, choice=wd_o)
    for l_o in linkage_options:
        Z = linkage(dist_mat, method=l_o)
        for crit_o in criterion_options:
            clone_clusters = fcluster(Z, 5, criterion=crit_o)
            score = adjusted_rand_score(df_truth['celltype'].tolist(), clone_clusters.tolist())
            score_dict[(wd_o,l_o,crit_o)] = score

Out of 66430 clonal distances, 0 are noninformative
Out of 66430 clonal distances, 0 are noninformative


#### Look through sorted values

In [21]:
listofTuples = sorted(score_dict.items() , reverse=True, key=lambda x: x[1])
for elem in listofTuples :
    print(elem[0] , "\t" , elem[1])

('energy', 'ward', 'maxclust') 	 0.7899530119959177
('wasserstein', 'ward', 'maxclust') 	 0.7885719124781362
('wasserstein', 'median', 'maxclust') 	 0.7616486910947495
('energy', 'average', 'maxclust') 	 0.7502881477816572
('energy', 'weighted', 'maxclust') 	 0.7412519902286092
('wasserstein', 'centroid', 'maxclust') 	 0.7294044632136572
('energy', 'complete', 'maxclust') 	 0.7289260391327324
('wasserstein', 'average', 'maxclust') 	 0.7105797278643441
('wasserstein', 'complete', 'maxclust') 	 0.6990144268531757
('wasserstein', 'weighted', 'maxclust') 	 0.5355769532850246
('energy', 'median', 'maxclust') 	 0.5007467235162486
('energy', 'centroid', 'maxclust') 	 0.47804527941231256
('energy', 'single', 'maxclust') 	 0.18600624102198973
('energy', 'ward', 'distance') 	 0.00014879882749808688
('energy', 'single', 'distance') 	 0.00014879882749808688
('energy', 'average', 'distance') 	 0.00014879882749808688
('energy', 'complete', 'distance') 	 0.00014879882749808688
('energy', 'weighted', 

### MNN Trials

In [29]:
linkage_options = ["ward", "single", "average", "complete", "weighted", "centroid", "median"]
#criterion_options = ["inconsistent", "distance", "maxclust", "monocrit", "maxclust_monocrit"]
criterion_options = ["inconsistent", "distance", "maxclust"]

score_dict = {}

for rad in [x / 10.0 for x in range(2, 20, 2)]:
    dist_mat = getdistance_mnn(clonegrouping_spmtx, coords_mtx, time_vec, rad)
    for l_o in linkage_options:
        Z = linkage(dist_mat, method=l_o)
        for crit_o in criterion_options:
            clone_clusters = fcluster(Z, 5, criterion=crit_o)
            score = adjusted_rand_score(df_truth['celltype'].tolist(), clone_clusters.tolist())
            score_dict[(rad,l_o,crit_o)] = score

In [30]:
listofTuples = sorted(score_dict.items() , reverse=True, key=lambda x: x[1])
for elem in listofTuples :
    print(elem[0] , "\t" , elem[1])

(1.2, 'centroid', 'maxclust') 	 0.03131186993903108
(1.2, 'median', 'maxclust') 	 0.03131186993903108
(1.2, 'ward', 'maxclust') 	 0.028912952393654295
(1.2, 'complete', 'maxclust') 	 0.028912952393654295
(1.2, 'weighted', 'maxclust') 	 0.028912952393654295
(1.2, 'ward', 'distance') 	 0.028796796354170623
(1.2, 'average', 'maxclust') 	 0.028791956342469394
(1.8, 'centroid', 'maxclust') 	 0.025611447811894922
(1.8, 'median', 'maxclust') 	 0.025611447811894922
(1.8, 'average', 'maxclust') 	 0.022357688273493243
(1.4, 'centroid', 'maxclust') 	 0.020748127573613523
(1.4, 'median', 'maxclust') 	 0.020748127573613523
(1.6, 'centroid', 'maxclust') 	 0.020276713949516283
(1.6, 'median', 'maxclust') 	 0.020276713949516283
(1.8, 'weighted', 'maxclust') 	 0.019733288429377296
(1.8, 'ward', 'distance') 	 0.019396314193171865
(1.4, 'ward', 'maxclust') 	 0.01854636814139023
(1.4, 'average', 'maxclust') 	 0.01854636814139023
(1.4, 'complete', 'maxclust') 	 0.01854636814139023
(1.4, 'weighted', 'maxclu

In [52]:
%run megatron_ak.ipynb
linkage_options = ["ward", "single", "average", "complete", "weighted", "centroid", "median"]
mnn_options = ["connectivity"]
#criterion_options = ["inconsistent", "distance", "maxclust", "monocrit", "maxclust_monocrit"]
criterion_options = ["inconsistent", "distance", "maxclust"]

score_dict = {}
for rad in [x / 10.0 for x in range(2, 20, 2)]:
    for mnn_o in mnn_options:
        dist_mat = getdistance_mnn(clonegrouping_spmtx, coords_mtx, time_vec, rad, mode=mnn_o)
        for l_o in linkage_options:
            Z = linkage(dist_mat, method=l_o)
            for crit_o in criterion_options:
                clone_clusters = fcluster(Z, 5, criterion=crit_o)
                score = adjusted_rand_score(df_truth['celltype'].tolist(), clone_clusters.tolist())
                score_dict[(rad,mnn_o,l_o,crit_o)] = score
listofTuples = sorted(score_dict.items() , reverse=True, key=lambda x: x[1])
for elem in listofTuples :
    print(elem[0] , "\t" , elem[1])

IndexError: row index (874) out of range